In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [3]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Drop Unnecessary Columns
#Row Number #CustomerId #Surname
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# Check for Null Value in The Data
df.isnull().sum() # No Null Values Present in Any Rows

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

### Split Train Test Data
Only After that apply Standard Scaler, Label Encoder, One Hot encoding

In [6]:
#X, y Data
X = df.drop(['Exited'], axis = 1)
y = df['Exited']

#Train Test Split
from sklearn.model_selection import train_test_split
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

### Gender and Geography Columns
Label Encoder for Gender Column-->Binary Class   
One Hot Encoder for Geography Column--> Multiclass

In [7]:
# Two Columns 'Gender' & 'Geography' need to conver in Numerical values
# Gender--> Label Encoder
# Geography--> One Hot Encoder

from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()

# Training Data
X_train_full['Gender'] = label.fit_transform(X_train_full['Gender'])
X_train_full['Gender'].value_counts()

# Testing Data
X_test['Gender'] = label.fit_transform(X_test['Gender'])
X_test['Gender'].value_counts()

Gender
1    1095
0     905
Name: count, dtype: int64

In [8]:
# One Hot Encoding for Geography
from sklearn.preprocessing import OneHotEncoder
OHE = OneHotEncoder(sparse_output=False)

geo_OneHot_encode = OHE.fit_transform(X_train_full[['Geography']])

#Convert geo_OneHot_encode to DataFrame
geo_encode_data = pd.DataFrame(geo_OneHot_encode, columns = OHE.get_feature_names_out(['Geography']))
geo_encode_data

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,1.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
7995,1.0,0.0,0.0
7996,1.0,0.0,0.0
7997,1.0,0.0,0.0
7998,1.0,0.0,0.0


In [1]:
# Reset Index of geo_encode_data and X_train_full

geo_encode_data.reset_index(drop=True, inplace=True)
X_train_full.reset_index(drop=True, inplace=True)

# Concat OHE DataFrame with Main Data
X_train_full = pd.concat([X_train_full.drop(['Geography'], axis = 1), geo_encode_data], axis = 1)

# pd.concat([Original DF, NEW DF], axis = 1)
#X_train_full = pd.concat([X_train_full, geo_encode_data], axis = 1)

NameError: name 'geo_encode_data' is not defined

In [10]:
X_train_full.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,686,1,32,6,0.00,2,1,1,179093.26,1.0,0.0,0.0
1,632,1,42,4,119624.60,2,1,1,195978.86,0.0,1.0,0.0
2,559,1,24,3,114739.92,1,1,0,85891.02,0.0,0.0,1.0
3,561,0,27,9,135637.00,1,1,0,153080.40,1.0,0.0,0.0
4,517,1,56,9,142147.32,1,0,0,39488.04,1.0,0.0,0.0


In [11]:
# Test Data
# One Hot Encoding on Geography Column
 
geo_OHE_test = OHE.fit_transform(X_test[['Geography']]) # this will be in array
#convert into Data Frame

geo_OHE_test_Data = pd.DataFrame(geo_OHE_test, columns = OHE.get_feature_names_out(['Geography']))
geo_OHE_test_Data

# Reset Index
geo_OHE_test_Data.reset_index(drop = True, inplace = True)
X_test.reset_index(drop = True, inplace = True)

# Concat Main Data with OHE_test
X_test = pd.concat([X_test.drop(['Geography'], axis = 1), geo_OHE_test_Data], axis = 1)
X_test.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,596,1,32,3,96709.07,2,0,0,41788.37,0.0,1.0,0.0
1,623,1,43,1,0.00,2,1,1,146379.30,1.0,0.0,0.0
2,601,0,44,4,0.00,2,1,0,58561.31,0.0,0.0,1.0
3,506,1,59,8,119152.10,2,1,1,170679.74,0.0,1.0,0.0
4,560,0,27,7,124995.98,1,1,1,114669.79,0.0,0.0,1.0


### Standardization

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_full_scaled = scaler.fit_transform(X_train_full)

#now split splited train data into train and Validation data
X_test_scaled = scaler.fit_transform(X_test)

In [13]:
# Pickle File 
import pickle
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label, file)

with open('one_hot_encoding_geo.pkl', 'wb') as file:
    pickle.dump(OHE, file)

In [14]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

### ANN Training

In [23]:
import tensorflow as tf

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [26]:
models = Sequential([
    Dense(64, activation = 'relu', input_shape = (X_train_full_scaled.shape[1], )), # Hidden Layer 1
    Dense(32, activation = 'relu'), # Hidden Layer 2
    Dense(1, activation = 'sigmoid') # output layer with two classes (binary class)
])


In [28]:
models.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
# Compile ANN
models.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [47]:
# TensorBoard
log_dir = "log/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callbacks = TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [61]:
# Early Stopping
early_stopping_callbacks = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True) 

In [62]:
 # Training
history = models.fit(
    X_train_full_scaled, y_train_full, validation_data = (X_test_scaled, y_test), epochs = 100,
    callbacks = [tensorflow_callbacks, early_stopping_callbacks]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2275 - accuracy: 0.9035 - val_loss: 0.4243 - val_accuracy: 0.8335
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2267 - accuracy: 0.9010 - val_loss: 0.4219 - val_accuracy: 0.8435
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2238 - accuracy: 0.9084 - val_loss: 0.4218 - val_accuracy: 0.8445
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2264 - accuracy: 0.9035 - val_loss: 0.4287 - val_accuracy: 0.8435
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2250 - accuracy: 0.9046 - val_loss: 0.4278 - val_accuracy: 0.8385
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2248 - accuracy: 0.9061 - val_loss: 0.4290 - val_accuracy: 0.8380
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2264 - accuracy: 0.9053 - val_loss: 0.4298 - val_accuracy: 0.8395

In [63]:
# save the model
models.save('model.h5')

In [64]:
# Load Tensorboard
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [65]:
%tensorboard --logdir log/fit20240912-225945

Reusing TensorBoard on port 6008 (pid 1216), started 0:07:30 ago. (Use '!kill 1216' to kill it.)